In [ ]:
from whampy import SkySurvey
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from scipy.interpolate import interp1d

from modspectra.cube import EmissionCube

import astropy.units as u
import astropy.constants as c
from astropy.coordinates import Angle

from astropy.coordinates import SkyCoord
from scipy.stats import median_absolute_deviation as mad

from matplotlib.colors import LogNorm
from scipy import stats

import cartopy.crs as ccrs
from whampy.scaleHeight import fit_scale_heights

from dustmaps.bayestar import BayestarQuery
from dustmaps.marshall import MarshallQuery

from kd import kd_utils
from kd import reid14_rotcurve
from kd import pdf_kd

import glob
import pickle
from whampy.lbvTracks import get_lbv_track

pal = sns.color_palette("colorblind")
%matplotlib notebook

sns.palplot(pal)

# Summary of Notebook:

## Calculate scale heights for HA and HI

Note: running notebook as is will write 100's of files used to make animations. 
Comment out kwarg:'fig_names' to avoid doing this.

In [ ]:
ha = SkySurvey()
bayestar = BayestarQuery()
marshall = MarshallQuery()

In [ ]:
# Load Data
hi_filename = "/Users/dk/Data/hi4pi/CAR.fits"
hi_cube = EmissionCube.read(hi_filename)

# Extract 1-D Coordinates
_,_,hi_lon = hi_cube.world[500,500,:]
_,hi_lat,_ = hi_cube.world[500,:,500]
hi_vel,_,_ = hi_cube.world[:,500,500]

# latitude Mask for All Arms
lat_mask = np.where(np.abs(hi_lat) < 30*u.deg)[0]

# Velocity Mask for All Arms
vel_mask = hi_vel > -150 * u.km/u.s
vel_mask &= hi_vel < 150 * u.km/u.s

vel_mask = np.where(vel_mask)[0]

# Restrict HI to lat and vel masks
hi_arms = hi_cube[vel_mask[0]:vel_mask[-1], lat_mask[0]:lat_mask[-1], :]


def apply_spiral_mask(cube, track, brange = None, vrange = None, lrange = None, 
                      vel_thickness = None, wrap_at_180 = True):
    """
    Applies spiral arm mask to SpectralCube
    
    Parameters
    ----------
    cube: `spectral_cube.SpectralCube` or `modpsectra.cube.EmissionCube`
        Spectral cube to apply spiral mask to
    track: `np.array`
        spiral arm track in lbv_RBD
    brange: `u.Quantity`
        latitude range (deg)
    vrange: `u.Quantity`
        velocity range (km/s)
    lrange: `u.Quantity`
        longitude range (deg)
    vel_thickness: `u.Quantity`
        thickness of velocity window
    wrap_at_180: `bool`
        where to wrap longitude
    """
    
    if wrap_at_180:
        wrap_at = "180d"
    else:
        wrap_at = "360d"
            
    # Set up defaults
    if brange is None:
        brange = [-40,40] * u.deg
    elif not hasattr(brange, "unit"):
        brange *= u.deg
    
    if vrange is None:
        vrange = [-150,150] * u.km/u.s
    elif not hasattr(vrange, "unit"):
        vrange *= u.km/u.s
        
    if vel_thickness is None:
        vel_thickness = 16*u.km/u.s
    elif not hasattr(vel_thickness, "unit"):
        vel_thickness *= u.km/u.s
    
    # Extract cube longitudes, latitudes, and velocity
    (x,y,z)  = cube.shape
    _,_,cube_lon = cube.world[int(x/2),int(y/2),:]
    _,cube_lat,_ = cube.world[int(x/2),:,int(z/2)]
    cube_vel,_,_ = cube.world[:,int(y/2),int(z/2)]
    
    #Extract relevant track information
    lon = track[:,0] * u.deg
    velocity = track[:,2] * u.km/u.s
    distance = track[:,-1] * u.kpc
    
    # Remove nan values in lon_axis
    lon_nan = np.isnan(cube_lon)
    lon_inds = np.arange(len(cube_lon))
    
    # mask for arm
    if lrange is None:
        lon_mask = Angle(cube_lon[~lon_nan]).wrap_at(wrap_at) <= Angle(lon).wrap_at(wrap_at).max()
        lon_mask &= Angle(cube_lon[~lon_nan]).wrap_at(wrap_at) >= Angle(lon).wrap_at(wrap_at).min()
    else:
        lon_mask = Angle(cube_lon[~lon_nan]).wrap_at(wrap_at) <= Angle(lrange[1]).wrap_at(wrap_at)
        lon_mask &= Angle(cube_lon[~lon_nan]).wrap_at(wrap_at) >= Angle(lrange[0]).wrap_at(wrap_at)
    lon_mask = np.where(lon_mask)[0]
    lon_mask_inds = lon_inds[~lon_nan]
    lon_mask_inds = lon_mask_inds[lon_mask]
    lat_mask = np.where((cube_lat <= brange[1]) & (cube_lat >= brange[0]))[0]
    vel_mask = np.where((cube_vel <= vrange[1]) & (cube_vel >= vrange[0]))[0]
    
    cube_cut = cube[vel_mask[0]:vel_mask[-1], lat_mask[0]:lat_mask[-1], lon_mask_inds[0]:lon_mask_inds[-1]]
    
    _,_,cut_lon = cube_cut.world[0,0,:]
    _,cut_lat,_ = cube_cut.world[0,:,0]
    cut_vel,_,_ = cube_cut.world[:,0,0]
    
    track_lv_interpolate = interp1d(lon, velocity)
    track_ld_interpolate = interp1d(lon, distance)
    cube_central_velocity = track_lv_interpolate(cut_lon)
    cube_distance = track_ld_interpolate(cut_lon)
    
    dv = cut_vel[1] - cut_vel[0]
    channel_thickness = np.round((vel_thickness / dv).decompose())
    
    mask = np.zeros(cube_cut.shape, dtype = bool)

    for ell, vel_cen in enumerate(cube_central_velocity):
        ind = np.argmin(np.abs((cut_vel - vel_cen*u.km/u.s).decompose()))
        mask[int(ind-channel_thickness/2):int(ind+channel_thickness/2),:,ell] =  True
        
    cut_cube_masked = EmissionCube(data = cube_cut.with_mask(mask).filled_data[:,:,:], 
                                   wcs = cube_cut.wcs)
    cut_cube_masked.central_velocity = cube_central_velocity * u.km/u.s
    cut_cube_masked.distance = cube_distance * u.kpc
    
    return cut_cube_masked


def fit_hi_scale_heights(hi_arm, longitude_width = None, 
                         min_lat = None, max_lat = None, 
                         return_smoothed = True, 
                         smoothed_width = None, 
                         fig_names = None, 
                         xlim = None, ylim = None):
    """
    Fit HI Column Density Scale Heights
    
    Parameters
    ----------
    hi_arm: `EmissionCube`
        hi EmissionCube with spiral arm masks applied
    longitude_width: `u.Quantity`
        width of longitude slices to use
    min_lat: `u.Quantity`
        minimum latitude to fit
    max_lat: `u.Quantity`
        maximum latitude to fit
    return_smoothed: `bool`
        if True, returns smoothed longitude and slope estimates
    smoothed_width: `u.Quantity`
        width to smooth data to in longitude
    fig_names: `str`
        if provided, saves figures following this name
    """
    if longitude_width is None:
        longitude_width = 1 * u.deg
    elif not hasattr(longitude_width, "unit"):
        longitude_width *= u.deg
        
    if min_lat is None:
        min_lat = 2 * u.deg
    elif not hasattr(longitude_width, "unit"):
        min_lat *= u.deg
        
    if max_lat is None:
        max_lat = 20 * u.deg
    elif not hasattr(longitude_width, "unit"):
        min_lat *= u.deg
        
    if smoothed_width is None:
        smoothed_width = 5*u.deg
    elif not hasattr(smoothed_width, "unit"):
        smoothed_width *= u.deg
        
    _,_,lons = hi_arm.world[0,0,:]
    _,lats,_ = hi_arm.world[0,:,0]
    vels,_,_ = hi_arm.world[:,0,0]
    dl = lons[1] - lons[0]
    step_width = int(np.round(np.abs(longitude_width/2. / dl)).value)
    
    lon_length = len(lons)
    lon_slices = np.arange(step_width, lon_length - step_width)
    
    slopes_hi_pos = []
    slopes_hi_neg = []
    intercept_hi_pos = []
    intercept_hi_neg = []
    longitude_center_hi = []
    distance_center_hi = []
    
    hi_arm_moment = hi_arm.moment().to(u.K * u.km/u.s)
    
    y_min = np.tan(min_lat)
    y_max = np.tan(max_lat)
    

    for ell2, lon_slice in enumerate(lon_slices):
        
        xx = np.tile(np.tan(lats), step_width * 2)
        yyy = np.zeros_like(xx).reshape(step_width*2,-1)
        for ell in range(step_width*2):
            slice_idx = lon_slice - step_width + ell
            yyy[ell,:] = (hi_arm_moment[:,slice_idx].value * 1.82e18 * u.cm**-2).value
            
        yy = np.log(yyy.flatten())
        
        
        siegel_result_pos = stats.siegelslopes(yy[(xx > y_min) & (xx < y_max) & ~np.isnan(yy)],
                                                   xx[(xx > y_min) & (xx < y_max) & ~np.isnan(yy)])
        siegel_result_neg = stats.siegelslopes(yy[(xx < -y_min) & (xx > -y_max) & ~np.isnan(yy)],
                                               xx[(xx < -y_min) & (xx > -y_max) & ~np.isnan(yy)])
        
        slopes_hi_pos.append(siegel_result_pos[0])
        slopes_hi_neg.append(siegel_result_neg[0])
        intercept_hi_pos.append(siegel_result_pos[1])
        intercept_hi_neg.append(siegel_result_neg[1])
        longitude_center_hi.append(lons[lon_slice].value)
        distance_center_hi.append(hi_arm.distance.value[lon_slice])
        
#         if fig_names is not None:
#             figure_name = "{0}_{1}.png".format(fig_names, ell2)
#             if xlim is None:
#                 xlim = np.array([-0.9, 0.9])
#             if ylim is None:
#                 ylim = np.array([-4.6, 3.2])
                
#             ax.scatter(xx, 
#                        yy, 
#                        color ="k",
#                        alpha = 0.8)
#             fig = plt.figure()
#             ax = fig.add_subplot(111)
#             ax2 = ax.twiny()
        
    results = {
        "median_longitude":longitude_center_hi*u.deg,
        "slopes_pos":np.array(slopes_hi_pos),
        "slopes_neg":np.array(slopes_hi_neg),
        "intercept_pos":np.array(intercept_hi_pos),
        "intercept_neg":np.array(intercept_hi_neg),
        "median_distnace":distance_center_hi*u.kpc
    }
    
    if return_smoothed:
        results["smoothed_longitude"] = np.arange(np.min(longitude_center_hi), 
            np.max(longitude_center_hi), 0.25)
        distance_interp = interp1d(longitude_center_hi, distance_center_hi)
        results["smoothed_distance"] = distance_interp(results["smoothed_longitude"])
        smoothed_slope_pos = np.zeros((3,len(results["smoothed_longitude"])))
        smoothed_slope_neg = np.zeros((3,len(results["smoothed_longitude"])))
        for ell,lon in enumerate(results["smoothed_longitude"]):
            smoothed_slope_pos[:,ell] = np.percentile(np.array(slopes_hi_pos)[(longitude_center_hi * u.deg <= lon*u.deg + smoothed_width/2) & 
                                                             (longitude_center_hi * u.deg > lon*u.deg - smoothed_width/2)], 
                                               (10, 50, 90))
            smoothed_slope_neg[:,ell] = np.percentile(np.array(slopes_hi_neg)[(longitude_center_hi * u.deg <= lon*u.deg + smoothed_width/2) & 
                                                             (longitude_center_hi * u.deg > lon*u.deg - smoothed_width/2)], 
                                               (10, 50, 90))

        results["smoothed_slopes_pos"] = smoothed_slope_pos
        results["smoothed_slopes_neg"] = smoothed_slope_neg
        
    return results
    



In [ ]:
track_dict = {
    "3kf":"3kF_lbvRBD.dat", "3kpc_far":"3kF_lbvRBD.dat", 
    "3kn":"3kN_lbvRBD.dat", "3kpc_near":"3kN_lbvRBD.dat", 
    "aqr":"AqR_lbvRBD.dat", "aquila_rift":"AqR_lbvRBD.dat", 
    "aqs":"AqS_lbvRBD.dat", "aquila_spur":"AqS_lbvRBD.dat", 
    "cnn":"CnN_lbvRBD.dat", "connecting_near":"CnN_lbvRBD.dat", 
    "cnx":"CnX_lbvRBD.dat", "connecting_extension":"CnX_lbvRBD.dat", 
    "crn":"CrN_lbvRBD.dat", "carina_near":"CrN_lbvRBD.dat", 
    "crf":"CrF_lbvRBD.dat", "carina_far":"CrF_lbvRBD.dat", 
    "ctn":"CtN_lbvRBD.dat", "centaurus_near":"CtN_lbvRBD.dat", 
    "ctf":"CtF_lbvRBD.dat", "centaurus_far":"CtF_lbvRBD.dat", 
    "loc":"Loc_lbvRBD.dat", "local":"Loc_lbvRBD.dat", 
    "los":"LoS_lbvRBD.dat", "local_spur":"LoS_lbvRBD.dat", 
    "nor":"Nor_lbvRBD.dat", "norma":"Nor_lbvRBD.dat", 
    "osc":"OSC_lbvRBD.dat", "outer_centaurus":"OSC_lbvRBD.dat", 
        "outer_scutum":"OSC_lbvRBD.dat", "outer_scutum_centaurus":"OSC_lbvRBD.dat", 
    "outer":"Out_lbvRBD.dat", "out":"Out_lbvRBD.dat", 
    "per":"Per_lbvRBD.dat", "perseus":"Per_lbvRBD.dat", 
    "scn":"ScN_lbvRBD.dat", "scutum_near":"ScN_lbvRBD.dat", 
    "scf":"ScF_lbvRBD.dat", "scutum_far":"ScF_lbvRBD.dat", 
    "sgn":"SgN_lbvRBD.dat", "sagittarius_near":"SgN_lbvRBD.dat",
    "sgf":"SgF_lbvRBD.dat", "sagittarius_far":"SgF_lbvRBD.dat"
}



# Near Carina Arm

In [ ]:
# Check Data
key = "crn"



track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["near"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["near"] - result["near_err_neg"], 
                result["near"] + result["near_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)

ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
track_kinematic = np.copy(track[(track[:,0] < 350),:])

distance = ([result["near_err_neg"][track[:,0] < 350], 
             result["near"][track[:,0] < 350], 
             result["near_err_pos"][track[:,0] < 350]])

rgal = ([result["Rgal_err_neg"][track[:,0] < 350], 
             result["Rgal"][track[:,0] < 350], 
             result["Rgal_err_pos"][track[:,0] < 350]])


track_kinematic[:,-1] = result["near"][track[:,0] < 350]
track_kinematic[:,-2] = result["Rgal"][track[:,0] < 350]


crn_ha = ha.get_spiral_slice(track = track_kinematic, brange = [-40,40])
(crn_ha_dered, crn_ha_df, \
    crn_ha_masks) = crn_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = marshall)



In [ ]:
# Fit Data

crn_ha_fit_results = fit_scale_heights(crn_ha_dered, 
                                crn_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/CarinaNear_ScaleHeights_width1_dered_marshall", 
                                deredden = True)

In [ ]:
#ffmpeg

!ffmpeg -r 24 -i Figures/Animations/CarinaNear_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/CarinaNear_ScaleHeights_width1_dered_marshall.mp4



In [ ]:
fit_results = crn_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
                 color = pal[3], 
                 alpha = 0.2)

# ax.set_ylim(0, 2.0)
# ax.set_xlim(170, 90)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Carina Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
crn_hi = apply_spiral_mask(hi_cube, crn_ha.lbv_RBD_track)

In [ ]:
crn_hi_fit_results = fit_hi_scale_heights(crn_hi)

In [ ]:
# Save Results
with open("crn_fit_results.pkl", "wb") as f:
    pickle.dump((track_kinematic, distance, rgal, crn_ha_fit_results, crn_hi_fit_results), f)

In [ ]:
with open("crn_fit_results.pkl", "rb") as f:
    (crn_track, crn_ha_fit_results, crn_hi_fit_results) = pickle.load(f)

In [ ]:
hi_fit_results = crn_hi_fit_results
fit_results = crn_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)





lg = ax.legend(fontsize = 12, loc = 2)

ax.set_ylim(0, 0.5)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)
# ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Carina Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 1)


ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)


ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], label = "Distance", lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
         color = pal[3], alpha = 0.1)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
ax3.set_xlim(350, 300)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/NearCarina_Arm_HI_HA.png", transparent = True, dpi = 300)

# Far Carina Arm

In [ ]:
# Check Data
key = "crf"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["far"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["far"] - result["far_err_neg"], 
                result["far"] + result["far_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)

# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
track_kinematic = np.copy(track[(track[:,0] < 350),:])

distance = ([result["far_err_neg"][track[:,0] < 350], 
             result["far"][track[:,0] < 350], 
             result["far_err_pos"][track[:,0] < 350]])

rgal = ([result["Rgal_err_neg"][track[:,0] < 350], 
             result["Rgal"][track[:,0] < 350], 
             result["Rgal_err_pos"][track[:,0] < 350]])


track_kinematic[:,-1] = distance[1]
track_kinematic[:,-2] = rgal[1]


crf_ha = ha.get_spiral_slice(track = track_kinematic, brange = [-40,40])
(crf_ha_dered, crf_ha_df, \
    crf_ha_masks) = crf_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = marshall)



In [ ]:
# Fit Data

crf_ha_fit_results = fit_scale_heights(crf_ha_dered, 
                                crf_ha_masks, 
                                return_smoothed=True, 
#                                 fig_names = "Figures/AnimationsCarinaFar_ScaleHeights_width1_dered_marshall", 
                                deredden = True)

In [ ]:
#ffmpeg

!ffmpeg -r 24 -i Figures/Animations/CarinaFar_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/CarinaFar_ScaleHeights_width1_dered_marshall.mp4



In [ ]:
fit_results = crf_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
                 color = pal[3], 
                 alpha = 0.2)

ax.set_ylim(0, 10)
ax.set_xlim(320, 280)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Far Carina Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
crf_hi = apply_spiral_mask(hi_cube, crf_ha.lbv_RBD_track)

In [ ]:
crf_hi_fit_results = fit_hi_scale_heights(crf_hi)

In [ ]:
# Save Results
with open("crf_fit_results.pkl", "wb") as f:
    pickle.dump((track_kinematic, distance, rgal, crf_ha_fit_results, crf_hi_fit_results), f)

# # Read Results
# with open("crf_fit_results.pkl", "rb") as f:
#     (_, crf_ha_fit_results, crf_hi_fit_results) =  pickle.load(f)

In [ ]:
hi_fit_results = crf_hi_fit_results
fit_results = crf_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)





lg = ax.legend(fontsize = 12, loc = 9)

ax.set_ylim(0, 4)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)
# ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Far Carina Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 8)
ax.set_xlim(320, 280)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], label = "Distance", lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
         color = pal[3], alpha = 0.1)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
plt.tight_layout()

# plt.savefig("Figures/FarCarina_Arm_HI_HA.png", transparent = True, dpi = 300)

# Perseus Arm

In [ ]:
# Check Data
key = "per"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["far"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["far"] - result["far_err_neg"], 
                result["far"] + result["far_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)

# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
# per_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
# (per_ha_dered, per_ha_df, \
#     per_ha_masks) = per_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
#                                                          return_pandas_dataframe = True, 
#                                                          deredden = marshall)

per_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
(per_ha_dered, per_ha_df, \
    per_ha_masks) = per_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = bayestar)

In [ ]:
# # Fit Data

# per_ha_fit_results = fit_scale_heights(per_ha_dered, 
#                                 per_ha_masks, 
#                                 return_smoothed=True, 
#                                 fig_names = "Figures/Animations/Perseus_ScaleHeights_width1_dered_marshall", 
#                                 deredden = True)

# Fit Data

per_ha_fit_results = fit_scale_heights(per_ha_dered, 
                                per_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/Perseus_ScaleHeights_width1_dered_bayestar", 
                                deredden = bayestar)

In [ ]:
#ffmpeg

# !ffmpeg -r 24 -i Figures/Animations/Perseus_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/Perseus_ScaleHeights_width1_dered_marshall.mp4

!ffmpeg -r 24 -i Figures/Animations/Perseus_ScaleHeights_width1_dered_bayestar_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/Perseus_ScaleHeights_width1_dered_bayestar.mp4


In [ ]:
fit_results = per_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0, 5)
ax.set_xlim(220, 60)
# ax2.set_ylim(0, 6)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Perseus Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
fit_results = per_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)


ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0, 2)
ax.set_xlim(160, 80)
ax2.set_ylim(0, 6)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Perseus Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
per_hi = apply_spiral_mask(hi_cube, 
                           per_ha.lbv_RBD_track, 
                           wrap_at_180 = False, 
                           lrange = [60,180]*u.deg)

In [ ]:
per_hi_q3 = apply_spiral_mask(hi_cube, 
                           per_ha.lbv_RBD_track, 
                           wrap_at_180 = False, 
                           lrange = [180.00001,250]*u.deg)

In [ ]:
per_hi_fit_results = fit_hi_scale_heights(per_hi)

In [ ]:
per_hi_q3_fit_results = fit_hi_scale_heights(per_hi_q3)

In [ ]:
# Save Results
with open("per_fit_results_bayestar.pkl", "wb") as f:
    pickle.dump((track, per_ha_fit_results, per_hi_fit_results, per_hi_q3_fit_results), f)

In [ ]:
# Read Results
with open("per_fit_results_bayestar.pkl", "rb") as f:
    (_, _, per_hi_fit_results, per_hi_q3_fit_results) =  pickle.load(f)

In [ ]:
hi_fit_results = per_hi_fit_results
fit_results = per_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

hi_fit_results = per_hi_q3_fit_results

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)




lg = ax.legend(fontsize = 12, loc = 9)

ax.set_ylim(0, 1.5)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Perseus Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

neg_lon_mask = fit_results["smoothed_longitude"] < 173
ax.plot(fit_results["smoothed_longitude"][neg_lon_mask], 
        fit_results["smoothed_distance"][neg_lon_mask]/ fit_results["smoothed_slopes_neg_dr"][1,neg_lon_mask], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"][neg_lon_mask], 
        fit_results["smoothed_distance"][neg_lon_mask]/ fit_results["smoothed_slopes_neg_dr"][0,neg_lon_mask], 
                fit_results["smoothed_distance"][neg_lon_mask]/ fit_results["smoothed_slopes_neg_dr"][2,neg_lon_mask], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 2)

ax.set_ylim(0, 3)
ax.set_xlim(220, 60)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2.plot(fit_results["smoothed_longitude"], fit_results["smoothed_distance"], 
         color = pal[3], label = "Distance", lw = 2)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/Perseus_Arm_HI_HA.png", transparent = True, dpi = 300)

# Near Sagittarius Arm

In [ ]:
# Check Data
key = "sgn"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["near"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["near"] - result["near_err_neg"], 
                result["near"] + result["near_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)

# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
# sgn_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
# (sgn_ha_dered, sgn_ha_df, \
#     sgn_ha_masks) = sgn_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
#                                                          return_pandas_dataframe = True, 
#                                                          deredden = marshall)

sgn_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
(sgn_ha_dered, sgn_ha_df, \
    sgn_ha_masks) = sgn_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = bayestar)

In [ ]:
# Fit Data

# sgn_ha_fit_results = fit_scale_heights(sgn_ha_dered, 
#                                 sgn_ha_masks, 
#                                 return_smoothed=True, 
#                                 fig_names = "Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_marshall", 
#                                 deredden = True)

sgn_ha_fit_results = fit_scale_heights(sgn_ha_dered, 
                                sgn_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_bayestar", 
                                deredden = bayestar)

In [ ]:
#ffmpeg

# !ffmpeg -r 24 -i Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_marshall.mp4

!ffmpeg -r 24 -i Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_bayestar_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/SagittariusNear_ScaleHeights_width1_dered_bayestar.mp4


In [ ]:
fit_results = sgn_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0,1.5)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Sagittarius Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
sgn_hi = apply_spiral_mask(hi_cube, 
                           sgn_ha.lbv_RBD_track, 
                           wrap_at_180 = False)

In [ ]:
sgn_hi_fit_results = fit_hi_scale_heights(sgn_hi)

In [ ]:
# Save Results
with open("sgn_fit_results.pkl", "wb") as f:
    pickle.dump((track, sgn_ha_fit_results, sgn_hi_fit_results), f)

In [ ]:
# Save Results
with open("sgn_fit_results.pkl", "rb") as f:
    (_, _, sgn_hi_fit_results) = pickle.load(f)

In [ ]:
hi_fit_results = sgn_hi_fit_results
fit_results = sgn_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)




lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 0.75)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)


ax.set_title("Near Sagittarius Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

neg_lon_mask = fit_results["smoothed_longitude"] < 173
ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"][neg_lon_mask], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 2)

ax.set_ylim(0, 1.5)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], label = "Distance", lw = 2)

ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
# ax3.set_xlim(350, 300)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/NearSagittarius_Arm_HI_HA.png", transparent = True, dpi = 300)

# Far Sagittarius Arm

In [ ]:
# Check Data
key = "sgf"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["far"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["far"] - result["far_err_neg"], 
                result["far"] + result["far_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)
ax.invert_xaxis()
# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
sgf_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
(sgf_ha_dered, sgf_ha_df, \
    sgf_ha_masks) = sgf_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = bayestar)

In [ ]:
sgf_ha_fit_results = fit_scale_heights(sgf_ha_dered, 
                                sgf_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/SagittariusFar_ScaleHeights_width1_dered_bayestar", 
                                deredden = bayestar)

In [ ]:
!ffmpeg -r 24 -i Figures/Animations/SagittariusFar_ScaleHeights_width1_dered_bayestar_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/SagittariusFar_ScaleHeights_width1_dered_bayestar.mp4


In [ ]:
fit_results = sgf_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0,5)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Far Sagittarius Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
sgf_hi = apply_spiral_mask(hi_cube, 
                           sgf_ha.lbv_RBD_track, 
                           wrap_at_180 = False)

In [ ]:
sgf_hi_fit_results = fit_hi_scale_heights(sgf_hi)

In [ ]:
# Save Results
with open("sgf_fit_results.pkl", "wb") as f:
    pickle.dump((track, sgf_ha_fit_results, sgf_hi_fit_results), f)

In [ ]:
# Save Results
with open("sgf_fit_results.pkl", "rb") as f:
    (_, _, sgf_hi_fit_results) = pickle.load(f)

In [ ]:
hi_fit_results = sgf_hi_fit_results
fit_results = sgf_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)




lg = ax.legend(fontsize = 12, loc = 4)

ax.set_ylim(0, 1.5)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)


ax.set_title("Far Sagittarius Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 2)

ax.set_ylim(0, 3)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], label = "Distance", lw = 2)

ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
# ax3.set_xlim(350, 300)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/FarSagittarius_Arm_HI_HA.png", transparent = True, dpi = 300)

# Near Scutum Arm

In [ ]:
# Check Data
key = "scn"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["near"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["near"] - result["near_err_neg"], 
                result["near"] + result["near_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)
ax.invert_xaxis()
# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
scn_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
(scn_ha_dered, scn_ha_df, \
    scn_ha_masks) = scn_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = bayestar)

In [ ]:
scn_ha_fit_results = fit_scale_heights(scn_ha_dered, 
                                scn_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/ScutumNear_ScaleHeights_width1_dered_bayestar", 
                                deredden = bayestar)

In [ ]:
!ffmpeg -r 24 -i Figures/Animations/ScutumNear_ScaleHeights_width1_dered_bayestar_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/ScutumNear_ScaleHeights_width1_dered_bayestar.mp4


In [ ]:
fit_results = scn_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0,1.5)
ax.set_xlim(35, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Scutum Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
scn_hi = apply_spiral_mask(hi_cube, 
                           scn_ha.lbv_RBD_track, 
                           wrap_at_180 = False)

In [ ]:
scn_hi_fit_results = fit_hi_scale_heights(scn_hi)

In [ ]:
# Save Results
with open("scn_fit_results.pkl", "wb") as f:
    pickle.dump((track, scn_ha_fit_results, scn_hi_fit_results), f)

In [ ]:
hi_fit_results = scn_hi_fit_results
fit_results = scn_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)




lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 0.75)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)


ax.set_title("Near Scutum Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 3)

ax.set_ylim(0, 1.5)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], label = "Distance", lw = 2)

ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
ax3.set_xlim(33, 15)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/NearScutum_Arm_HI_HA.png", transparent = True, dpi = 300)

# Far Scutum Arm

In [ ]:
# Check Data
key = "scf"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["far"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["far"] - result["far_err_neg"], 
                result["far"] + result["far_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)
ax.invert_xaxis()
# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
scf_ha = ha.get_spiral_slice(track = track, brange = [-40,40])
(scf_ha_dered, scf_ha_df, \
    scf_ha_masks) = scf_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = bayestar)

In [ ]:
scf_ha_fit_results = fit_scale_heights(scf_ha_dered, 
                                scf_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/ScutumFar_ScaleHeights_width1_dered_bayestar", 
                                deredden = bayestar)

In [ ]:
!ffmpeg -r 24 -i Figures/Animations/ScutumFar_ScaleHeights_width1_dered_bayestar_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/ScutumFar_ScaleHeights_width1_dered_bayestar.mp4


In [ ]:
fit_results = scf_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], lw = 2)
# ax2.fill_between(track_kinematic[:,0], 
#                  distance[1] - distance[0], 
#                  distance[1] + distance[2], 
#                  color = pal[3], 
#                  alpha = 0.2)

ax.set_ylim(0,5)
ax.set_xlim(35, 20)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Far Scutum Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
scf_hi = apply_spiral_mask(hi_cube, 
                           scf_ha.lbv_RBD_track, 
                           wrap_at_180 = False)

In [ ]:
scf_hi_fit_results = fit_hi_scale_heights(scf_hi)

In [ ]:
# Save Results
with open("scf_fit_results.pkl", "wb") as f:
    pickle.dump((track, scf_ha_fit_results, scf_hi_fit_results), f)

In [ ]:
hi_fit_results = scf_hi_fit_results
fit_results = scf_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)




lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 1.5)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)


ax.set_title("Far Scutum Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 3)

ax.set_ylim(0, 3)
ax.set_xlim(50, 10)

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)

ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track[:,0], track[:,-1], 
         color = pal[3], label = "Distance", lw = 2)

ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
ax3.set_xlim(33, 15)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/FarScutum_Arm_HI_HA.png", transparent = True, dpi = 300)

# Near Centaurus Arm

In [ ]:
# Check Data
key = "ctn"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["near"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["near"] - result["near_err_neg"], 
                result["near"] + result["near_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)
ax.invert_xaxis()
# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
track_kinematic = np.copy(track[(track[:,0] < 350),:])

distance = ([result["near_err_neg"][track[:,0] < 350], 
             result["near"][track[:,0] < 350], 
             result["near_err_pos"][track[:,0] < 350]])

rgal = ([result["Rgal_err_neg"][track[:,0] < 350], 
             result["Rgal"][track[:,0] < 350], 
             result["Rgal_err_pos"][track[:,0] < 350]])


track_kinematic[:,-1] = result["near"][track[:,0] < 350]
track_kinematic[:,-2] = result["Rgal"][track[:,0] < 350]


ctn_ha = ha.get_spiral_slice(track = track_kinematic, brange = [-40,40])
(ctn_ha_dered, ctn_ha_df, \
    ctn_ha_masks) = ctn_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = marshall)



In [ ]:
ctn_ha_fit_results = fit_scale_heights(ctn_ha_dered, 
                                ctn_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/CentaurusNear_ScaleHeights_width1_dered_marshall", 
                                deredden = True)

In [ ]:
!ffmpeg -r 24 -i Figures/Animations/CentaurusNear_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/CentaurusNear_ScaleHeights_width1_dered_marshall.mp4


In [ ]:
fit_results = ctn_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
                 color = pal[3], 
                 alpha = 0.2)

# ax.set_ylim(0, 2.0)
# ax.set_xlim(170, 90)

ax.invert_xaxis()

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Centaurus Arm", fontsize = 14)

plt.tight_layout()

In [ ]:
ctn_hi = apply_spiral_mask(hi_cube, 
                           ctn_ha.lbv_RBD_track, 
                           wrap_at_180 = False)

ctn_hi_fit_results = fit_hi_scale_heights(ctn_hi)

In [ ]:
# Save Results
with open("ctn_fit_results.pkl", "wb") as f:
    pickle.dump((track_kinematic, distance, rgal, ctn_ha_fit_results, ctn_hi_fit_results), f)

In [ ]:
# with open("ctn_fit_results.pkl", "rb") as f:
#     (track_kinematic, ctn_ha_fit_results, ctn_hi_fit_results) = pickle.load(f)

In [ ]:
hi_fit_results = ctn_hi_fit_results
fit_results = ctn_ha_fit_results

fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"HI $b > 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][0,:], 
                hi_fit_results["smoothed_distance"]/ -hi_fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"HI $b < 0\degree$")
ax.fill_between(hi_fit_results["smoothed_longitude"], 
        hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][0,:], 
                hi_fit_results["smoothed_distance"]/ hi_fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)





lg = ax.legend(fontsize = 12, loc = 3)

ax.set_ylim(0, 0.75)
# ax.set_xlim(320, 282)

ax.set_ylabel("$H_{{N_{{HI}}}}$ (kpc)", fontsize = 12)
# ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Near Centaurus Arm", fontsize = 14)


ax = ax.twinx()

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"H$\alpha$ $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"H$\alpha$ $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)

lg = ax.legend(fontsize = 12, loc = 1)

ax.set_ylim(0, 1.5)


ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)


ax2 = fig.add_subplot(212, sharex = ax)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], label = "Distance", lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
         color = pal[3], alpha = 0.1)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax3 = ax2.twinx()
ax3.plot(track[:,0], track[:,2], 
         color = pal[2], label = "Velocity", lw = 2)
ax3.fill_between(track[:,0], 
                 track[:,2] - 8, 
                 track[:,2] + 8, 
                 color = pal[2], alpha = 0.1)

lh, ll = ax2.get_legend_handles_labels()
lh2, ll2 = ax3.get_legend_handles_labels()

ax3.legend(np.concatenate([lh, lh2]), np.concatenate([ll, ll2]), fontsize = 12)
ax3.set_ylabel("LSR Velocity (km/s)", fontsize = 12)
ax.set_xlim(305,350)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

plt.tight_layout()

# plt.savefig("Figures/NearCentaurus_Arm_HI_HA.png", transparent = True, dpi = 300)

# Far Centaurus Arm

In [ ]:
# Check Data
key = "ctf"

track = get_lbv_track(filename = "Reid16_SpiralArms/{}".format(
    track_dict[key]))
filename = "KinematicDistanceData/{}_kd_data.pkl".format(key)
with open(filename, "rb") as f:
    result = pickle.load(f)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(track[:,0], result["far"], 
        color = pal[0], 
        lw = 2, 
        label = "Kinematic Distance")
ax.fill_between(track[:,0], 
                result["far"] - result["far_err_neg"], 
                result["far"] + result["far_err_neg"], 
                color = pal[0], alpha = 0.2)

ax.plot(track[:,0], track[:,-1], 
        color = pal[1], 
        lw = 2, 
        label = "Reid Distance")

ax.set_xlabel("Galactic Longitude (deg)", fontsize = 12)
ax.set_ylabel("Distance (kpc)")

lg = ax.legend(fontsize = 12)
ax.invert_xaxis()
# ax.set_ylim(0, 2)

ax.set_title("Track Name: {}".format(key), fontsize = 12)
plt.tight_layout()

In [ ]:
track_kinematic = np.copy(track[(track[:,0] < 350),:])

distance = ([result["far_err_neg"][track[:,0] < 350], 
             result["far"][track[:,0] < 350], 
             result["far_err_pos"][track[:,0] < 350]])

rgal = ([result["Rgal_err_neg"][track[:,0] < 350], 
             result["Rgal"][track[:,0] < 350], 
             result["Rgal_err_pos"][track[:,0] < 350]])


track_kinematic[:,-1] = result["far"][track[:,0] < 350]
track_kinematic[:,-2] = result["Rgal"][track[:,0] < 350]


ctf_ha = ha.get_spiral_slice(track = track_kinematic, brange = [-40,40])
(ctf_ha_dered, ctf_ha_df, \
    ctf_ha_masks) = ctf_ha.get_scale_height_data(longitude_mask_width = 1 * u.deg, 
                                                         return_pandas_dataframe = True, 
                                                         deredden = marshall)



In [ ]:
ctf_ha_fit_results = fit_scale_heights(ctf_ha_dered, 
                                ctf_ha_masks, 
                                return_smoothed=True, 
                                fig_names = "Figures/Animations/CentaurusFar_ScaleHeights_width1_dered_marshall", 
                                deredden = True)

In [ ]:
!ffmpeg -r 24 -i Figures/Animations/CentaurusFar_ScaleHeights_width1_dered_marshall_%d.png -y -an -vf scale=1024:-2,format=yuv420p Figures/Animations/CentaurusFar_ScaleHeights_width1_dered_marshall.mp4


In [ ]:
fit_results = ctf_ha_fit_results


fig = plt.figure(figsize = (9,8))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex = ax)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][1,:], 
        color = "r", lw = 2, ls = "-", label = r"$b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][1,:], 
        color = "b", lw = 2, ls = "-", label = r"$b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg"][2,:], 
        color = "b", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][1,:], 
        color = "r", lw = 2, ls = ":", label = r"Extinction Corrected $b > 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][0,:], 
                fit_results["smoothed_distance"]/ -fit_results["smoothed_slopes_pos_dr"][2,:], 
        color = "r", alpha = 0.1)

ax.plot(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][1,:], 
        color = "b", lw = 2, ls = ":", label = r"Extinction Corrected $b < 0\degree$")
ax.fill_between(fit_results["smoothed_longitude"], 
        fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][0,:], 
                fit_results["smoothed_distance"]/ fit_results["smoothed_slopes_neg_dr"][2,:], 
        color = "b", alpha = 0.1)


lg = ax.legend(fontsize = 12)

ax2.plot(track_kinematic[:,0], distance[1], 
         color = pal[3], lw = 2)
ax2.fill_between(track_kinematic[:,0], 
                 distance[1] - distance[0], 
                 distance[1] + distance[2], 
                 color = pal[3], 
                 alpha = 0.2)

# ax.set_ylim(0, 2.0)
# ax.set_xlim(170, 90)

ax.invert_xaxis()

ax.set_ylabel("$H_{{n_{{e}}^2}}$ (kpc)", fontsize = 12)
ax2.set_ylabel("Distance (kpc)", fontsize = 12)
ax2.set_xlabel("Galactic Longitude (deg)", fontsize = 12)

ax.set_title("Far Centaurus Arm", fontsize = 14)

plt.tight_layout()